# tutorial 
- https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


# Example: An LSTM for Part-of-Speech Tagging

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])


# check parameters of model

In [15]:
help(model.parameters)

Help on method parameters in module torch.nn.modules.module:

parameters(recurse: bool = True) -> Iterator[torch.nn.parameter.Parameter] method of __main__.LSTMTagger instance
    Returns an iterator over module parameters.
    
    This is typically passed to an optimizer.
    
    Args:
        recurse (bool): if True, then yields parameters of this module
            and all submodules. Otherwise, yields only parameters that
            are direct members of this module.
    
    Yields:
        Parameter: module parameter
    
    Example::
    
        >>> # xdoctest: +SKIP("undefined vars")
        >>> for param in model.parameters():
        >>>     print(type(param), param.size())
        <class 'torch.Tensor'> (20L,)
        <class 'torch.Tensor'> (20L, 1L, 5L, 5L)



In [14]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[ 7.3412e-01,  8.0615e-04, -3.3758e-01,  6.2446e-01,  2.0486e-01,
         -1.0973e+00],
        [-9.2287e-01, -7.1421e-01,  9.6096e-01, -1.2692e+00, -1.8903e-01,
         -6.0069e-01],
        [ 6.4489e-01, -6.8376e-01,  2.1375e+00,  6.7106e-01,  3.9116e-01,
         -7.5065e-01],
        [ 4.5504e-01,  1.3352e+00, -9.5383e-01, -1.2280e+00,  3.4942e+00,
         -2.5017e+00],
        [-4.8974e-01, -5.7369e-02,  9.1785e-01, -1.1828e+00,  9.0761e-01,
          3.0940e-01],
        [ 1.4131e+00,  1.7109e+00,  1.4875e-01, -2.6694e+00,  2.0926e-01,
          2.0688e-01],
        [-9.3109e-02,  1.4724e-01,  1.4168e+00,  1.2175e+00,  1.1629e+00,
          9.5881e-01],
        [-6.4901e-01,  1.5777e+00,  1.4035e-01,  2.0601e+00, -3.1149e-01,
         -8.6065e-01],
        [-1.0945e+00,  1.2314e+00,  1.4436e+00, -7.7375e-01,  2.1218e-01,
         -5.7816e-01]], requires_grad=True)
Parameter containing:
tensor([[-0.3364, -0.2644,  0.5200, -0.1999,  0.1337,  0.1206]

In [17]:
for param in model.parameters():
    print(type(param), param.size())

<class 'torch.nn.parameter.Parameter'> torch.Size([9, 6])
<class 'torch.nn.parameter.Parameter'> torch.Size([24, 6])
<class 'torch.nn.parameter.Parameter'> torch.Size([24, 6])
<class 'torch.nn.parameter.Parameter'> torch.Size([24])
<class 'torch.nn.parameter.Parameter'> torch.Size([24])
<class 'torch.nn.parameter.Parameter'> torch.Size([3, 6])
<class 'torch.nn.parameter.Parameter'> torch.Size([3])


In [21]:
!pip install prettytable

In [22]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Mod name", "Parameters Listed"])
    t_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        t_params += param
    print(table)
    print(f"Sum of trained paramters: {t_params}")
    return t_params

count_parameters(model)

+------------------------+-------------------+
|        Mod name        | Parameters Listed |
+------------------------+-------------------+
| word_embeddings.weight |         54        |
|   lstm.weight_ih_l0    |        144        |
|   lstm.weight_hh_l0    |        144        |
|    lstm.bias_ih_l0     |         24        |
|    lstm.bias_hh_l0     |         24        |
|   hidden2tag.weight    |         18        |
|    hidden2tag.bias     |         3         |
+------------------------+-------------------+
Sum of trained paramters: 411


411

In [23]:
print(model)

LSTMTagger(
  (word_embeddings): Embedding(9, 6)
  (lstm): LSTM(6, 6)
  (hidden2tag): Linear(in_features=6, out_features=3, bias=True)
)


## empty-parameter model (not working)
- https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super(model, self).__init__()

# model = Model()

# for parameter in model.parameters():
#     print(parameter)